Dodanie bibliotek

In [2]:
import xgboost as xgb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

Wczytanie danych oraz prezentacja

In [3]:
dtrain =pd.read_csv("D:/ml/xgboost-main/data/ssh_logs/SSH.csv")
dtrain.head()

,user,is_private,is_failure,is_root,is_valid,not_valid_count,ip_failure,ip_success,no_failure,first,td,ts,target
0,kamran,1,1,0,1,0,1,0,2,1,0,1.544679e+09,0
1,kamran,1,1,0,1,0,2,0,3,0,18,1.544679e+09,0
2,student,1,0,0,1,0,0,1,0,0,133,1.544679e+09,0
3,kamran,1,1,0,1,0,1,1,2,0,58,1.544679e+09,0
4,kamran,1,1,0,1,0,2,1,3,0,29,1.544679e+09,0


Usuniecie danych uzytkownika skad pochodzi atak oraz danych ts 

In [4]:
dtrain = dtrain.drop(columns=['user', 'ts'])

dtrain.head()

,is_private,is_failure,is_root,is_valid,not_valid_count,ip_failure,ip_success,no_failure,first,td,target
0,1,1,0,1,0,1,0,2,1,0,0
1,1,1,0,1,0,2,0,3,0,18,0
2,1,0,0,1,0,0,1,0,0,133,0
3,1,1,0,1,0,1,1,2,0,58,0
4,1,1,0,1,0,2,1,3,0,29,0


y1 jako dane wyjsciowe oraz X1 jako dane testowe do uczenia sie

In [5]:
y1 = dtrain["target"]
X1 = dtrain.drop(columns=['target'])

In [6]:
dtrain.dtypes

is_private         int64
is_failure         int64
is_root            int64
is_valid           int64
not_valid_count    int64
ip_failure         int64
ip_success         int64
no_failure         int64
first              int64
td                 int64
target             int64
dtype: object

przygotowanie probek testowych oraz do uczenia modelu

In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    X1,
    y1,
    test_size=0.2)

print(X_train.shape)
print(X_test.shape)

(226, 10)
(57, 10)


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size=0.2, random_state=5)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(226, 10)
(226,)
(57, 10)
(57,)


Przygotowanie modelu

In [9]:
model = XGBClassifier()

model.fit(X_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [10]:
y_pred = model.predict(X_test)

In [11]:
accuracy = accuracy_score(y_test, y_pred) * 100
print("Accuracy of the Logistic Regression Model: ",accuracy)

Accuracy of the Logistic Regression Model:  96.49122807017544
